## 把mysql中保存的新闻存到es中

In [1]:
import pymysql
# 创建连接
cnx = pymysql.connect(host='localhost', user='root', password='123456')
cursor = cnx.cursor()

In [2]:
ids = []
def create_database():
    # 创建数据库
    cursor.execute("CREATE DATABASE IF NOT EXISTS IR_db")
    # 使用数据库
    cnx.select_db('IR_db')

def get_ids():
    # 创建SQL查询
    sql = "SELECT id FROM html"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids = [result[0] for result in results]
    return ids

def get_content(id):
    # 创建SQL查询
    sql = "SELECT * FROM html WHERE id = %s"
    # 执行查询
    cursor.execute(sql, (id,))
    # 获取结果
    result = cursor.fetchone()
    # 如果结果不为空，将结果转换为字典并返回，否则返回None
    if result:
        return {
            'id': result[0],
            'title': result[1],
            'content': result[2],
            'url': result[3],
            'date': result[4].strftime("%Y-%m-%d") if result[4] else None
        }
    else:
        return None

create_database()
ids = get_ids()
# get_content(1)

### 创建ES索引


In [3]:
from elasticsearch import Elasticsearch

# 实例化es
es = Elasticsearch(hosts="http://localhost:9200")
# 创建一个索引为test的索引
#es.indices.create(index='news')

### 读取数据插入ES

In [4]:
global_id = 11074
def insert_data_to_es(id):
    global global_id
    # 获取数据
    data = get_content(id)
    doc = {
        'web_id': data['id'], # 为了方便后面的更新操作，将id存储为web_id
        'title': data['title'],
        'content': data['content'],
        'url': data['url'],
        #'date': data['date'],
        'type': 'news',
        'pageRank': 0
    }
    # 插入数据
    es.index(index='web', id=global_id, document=doc)

def insert_data_to_es_all():
    global global_id
    for id in ids:
        insert_data_to_es(id)
        global_id += 1


In [5]:
insert_data_to_es_all()

C:\Users\89214\AppData\Local\Temp\ipykernel_32380\2658414841.py:16: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index='web', id=global_id, document=doc)


### 测试查询数据

In [6]:
es.get(index='web',id='11074')

C:\Users\89214\AppData\Local\Temp\ipykernel_32380\3752455278.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.get(index='web',id='11074')


ObjectApiResponse({'_index': 'web', '_type': '_doc', '_id': '11074', '_version': 3, '_seq_no': 24258, '_primary_term': 3, 'found': True, '_source': {'web_id': 1, 'title': '越南两位副总理辞去党内职务', 'content': '综合《日经亚洲》和越通社报道，当地时间12月30日，越南常务副总理范平明和负责防疫工作的副总理武德儋被要求辞去党内职务。\n《日经亚洲》指出，范平明作为中央政治局委员被赶出决策机构是极不寻常的。其需要对组织实施接越南公民回国航班工作中的丑闻负责。\n此前报道显示，新冠疫情期间，越南安排了约2000班海外救援包机，将滞留海外的越南公民接送回国，但很多人不得不高价购买机票，并经历繁琐手续才能搭乘航班。包括越南外交部长裴青山在内的多名官员近日因此受到处分。\n《日经亚洲》称，武德儋需要对“越亚案”负责。\n此前报道显示，越亚公司在Viet A试剂盒获得流通许可后，与越南卫生部门官员勾结，伪造文件、夸大试剂盒材料成本，将试剂盒的参考价抬高了45%。截至6月9日，已有超过60名与越亚案有关的人员被起诉及拘留，其中包括越共中央委员、河内市人民委员会主席（相当于市长）、原科技部长朱玉英和越共中央委员、卫生部长阮青龙。', 'url': 'https://www.guancha.cn/internation/2022_12_31_673623.shtml', 'type': 'news', 'pageRank': 0}})

In [7]:
def deal_bool_query(query):
    # 执行搜索
    response = es.search(
        index="news",
        body={
            "query": {
                "bool": {
                    "must": [
                        {
                            "multi_match": {
                                "query": query,
                                "fields": ["title", "content"]
                            }
                        }
                    ]
                }
            }
        }
    )
    # 获取搜索结果
    hits = response["hits"]["hits"]
    # 格式化搜索结果
    results = [
        {
            "id": hit["_id"],
            "title": hit["_source"]["title"],
            "content": hit["_source"]["content"],
            "url": hit["_source"]["url"],
            "date": hit["_source"]["date"]
        }
        for hit in hits
    ]
    return results

In [8]:
results = deal_bool_query("恒大")
for result in results:
    print(result['title'], result['date'])

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [news]', news, index_or_alias)